## SVM para classificação:

Entrada: um vetor unificado com as características das duas imagens sendo a última coluna 1 (mesma pessoa) ou 0 (pessoa diferente)

Passos: verificar vetor e definir se 1 ou 0, baseado na lógica da sequência de features

Saída: 1 ou 0

In [1]:
import pandas as pd
import numpy as np

## HOG

In [2]:
#carregar os dados

hog = pd.read_parquet('../../data/preprocessed/feature_matrix_hog.parquet', engine='pyarrow') 

In [3]:
hog.head()

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,match
0,2.306814,3.954538,3.624993,3.460221,2.636359,3.130676,2.636359,2.306814,1.647724,2.636359,...,0.295641,0.000000,0.295641,0.000000,0.000000,0.147820,0.000000,0.000000,0.147820,True
1,2.193303,2.976625,3.759947,3.603283,4.073276,4.073276,1.253316,1.409980,2.349967,0.939987,...,0.500357,0.333572,0.333572,0.000000,0.333572,0.000000,0.500357,0.000000,0.166786,True
2,3.875630,3.720604,3.100504,3.410554,3.100504,4.030655,1.860302,1.860302,1.550252,1.395227,...,0.482976,0.160992,0.321984,0.160992,0.321984,0.160992,0.160992,0.160992,0.160992,True
3,4.624305,3.545300,3.545300,2.466296,3.391157,2.928726,1.849722,2.158009,1.387291,1.695578,...,0.132290,0.132290,0.132290,0.000000,0.000000,0.000000,0.132290,0.000000,0.132290,True
4,2.934841,4.255520,4.549004,3.962035,3.521809,1.614163,1.614163,1.467421,0.733710,1.173936,...,0.501895,0.167298,0.167298,0.334597,0.167298,0.167298,0.000000,0.000000,0.334597,True


In [17]:
hog.shape

(3197, 61)

In [28]:
X = hog.iloc[:, 0:-1].values
y = hog.iloc[:, -1].values

In [29]:
X.shape

(3197, 60)

In [30]:
y.shape

(3197,)

In [21]:
y

array([True, True, True, ..., False, False, False], dtype=object)

In [25]:
y = [int(tag is True) for tag in y]

In [34]:
y = np.array(y)

In [35]:
y = 2*y-1

In [12]:
!pip install cvxopt

In [13]:
from cvxopt import matrix as cvxopt_matrix
from cvxopt import solvers as cvxopt_solvers

In [37]:
C = 1000
m, n = X.shape

y = y.reshape(-1,1) * 1.
X_dash = y * X
H = np.dot(X_dash , X_dash.T) * 1.

In [38]:
#Converting into cvxopt format - as previously
P = cvxopt_matrix(H)
q = cvxopt_matrix(-np.ones((m, 1)))
G = cvxopt_matrix(np.vstack((np.eye(m)*-1,np.eye(m))))
h = cvxopt_matrix(np.hstack((np.zeros(m), np.ones(m) * C)))
A = cvxopt_matrix(y.reshape(1, -1))
b = cvxopt_matrix(np.zeros(1))

#Run solver
sol = cvxopt_solvers.qp(P, q, G, h, A, b)
alphas = np.array(sol['x'])

     pcost       dcost       gap    pres   dres
 0: -1.3853e+06 -8.6594e+08  9e+08  3e-10  1e-07
 1: -1.5709e+06 -1.1923e+07  1e+07  1e-10  1e-07
 2: -1.9437e+06 -3.2118e+06  1e+06  5e-10  1e-07
 3: -2.6166e+06 -3.1189e+06  5e+05  5e-11  2e-07
 4: -2.7951e+06 -3.0453e+06  3e+05  2e-09  2e-07
 5: -2.8899e+06 -3.0016e+06  1e+05  1e-09  2e-07
 6: -2.9348e+06 -2.9804e+06  5e+04  1e-10  2e-07
 7: -2.9507e+06 -2.9714e+06  2e+04  8e-10  2e-07
 8: -2.9593e+06 -2.9662e+06  7e+03  7e-10  2e-07
 9: -2.9614e+06 -2.9645e+06  3e+03  3e-10  2e-07
10: -2.9625e+06 -2.9636e+06  1e+03  9e-10  2e-07
11: -2.9629e+06 -2.9633e+06  4e+02  6e-10  2e-07
12: -2.9630e+06 -2.9631e+06  7e+01  4e-10  2e-07
13: -2.9631e+06 -2.9631e+06  2e+01  2e-09  2e-07
14: -2.9631e+06 -2.9631e+06  3e+00  2e-09  2e-07
15: -2.9631e+06 -2.9631e+06  6e-02  4e-10  2e-07
16: -2.9631e+06 -2.9631e+06  6e-04  5e-10  2e-07
17: -2.9631e+06 -2.9631e+06  6e-06  6e-10  2e-07
18: -2.9631e+06 -2.9631e+06  6e-08  1e-09  2e-07
19: -2.9631e+06 -2.96

In [44]:
w = ((y * alphas).T @ X).reshape(-1,1)
S = (alphas > 1e-4).flatten()
b = y[S] - np.dot(X[S], w)

#Display results
print('Alphas = ',alphas[alphas > 1e-4])
print('w = ', w.flatten())
print('b = ', b[0])

Alphas =  [1000. 1000. 1000. ... 1000. 1000. 1000.]
w =  [ 0.09798541  0.22309909  0.30456065  0.42259339  0.34636078  0.1175113
  0.04521523  0.13845738  0.13588141 -0.09621655 -0.35189821 -0.04528985
  0.03543911 -0.34098364  0.21363465  0.14527136 -0.35864964  0.36436851
 -0.15327086 -0.0856962   0.05330152 -0.4121907  -0.42683544  0.18351369
  1.02959916  0.70508263  0.56399668  0.25539109  0.04668104 -0.74451809
  0.19389498 -0.02783845  0.07541202 -0.04998555  0.10113865  0.00287325
  0.18218848 -0.05385237  0.16767567 -0.21778907  0.27298719  0.33837374
 -0.13047347 -0.08157393 -0.12728968 -0.25715156  0.09744519  0.04842594
 -0.2582964   0.12489336  0.08253298 -0.14330269 -0.10427462 -0.64164057
 -0.21134168  0.23415632 -0.506652   -0.66449234  0.82825806  0.82324396]
b =  [-5.37662944]


In [41]:
w.shape

(60, 1)

In [43]:
X.shape

(3197, 60)

In [48]:
svm = np.matmul(X, w) + b[0]

In [50]:
y_predic = np.where(svm > 0, 1, -1).flatten()

In [52]:
y_predic

array([1, 1, 1, ..., 1, 1, 1])

In [53]:
from sklearn.metrics import classification_report

In [55]:
print(classification_report(y_pred=y_predic, y_true=y, labels=[-1,1]))

              precision    recall  f1-score   support

          -1       0.67      0.10      0.18      1598
           1       0.51      0.95      0.67      1599

    accuracy                           0.53      3197
   macro avg       0.59      0.53      0.42      3197
weighted avg       0.59      0.53      0.42      3197



## LBP

In [56]:
#carregar os dados

lbp = pd.read_parquet('../../data/preprocessed/feature_matrix_lbp.parquet', engine='pyarrow') 

In [57]:
lbp.head()

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,match
0,0.624000,0.0,0.0,0.0,0.673524,0.0,0.0,0.0,0.367238,0.0,...,0.502857,0.0,0.0,0.0,0.448762,0.0,0.0,0.0,0.417524,True
1,0.729143,0.0,0.0,0.0,0.736762,0.0,0.0,0.0,0.450286,0.0,...,0.271238,0.0,0.0,0.0,0.848762,0.0,0.0,0.0,0.412190,True
2,0.688000,0.0,0.0,0.0,0.857905,0.0,0.0,0.0,0.317714,0.0,...,0.561524,0.0,0.0,0.0,0.592000,0.0,0.0,0.0,0.550095,True
3,0.652190,0.0,0.0,0.0,0.876190,0.0,0.0,0.0,0.513524,0.0,...,0.228571,0.0,0.0,0.0,0.623238,0.0,0.0,0.0,0.579048,True
4,0.670476,0.0,0.0,0.0,0.649143,0.0,0.0,0.0,0.673524,0.0,...,0.369524,0.0,0.0,0.0,0.711619,0.0,0.0,0.0,0.518857,True


In [58]:
lbp.shape

(3197, 61)

In [59]:
X = lbp.iloc[:, 0:-1].values
y = lbp.iloc[:, -1].values

In [60]:
X.shape

(3197, 60)

In [61]:
y.shape

(3197,)

In [62]:
y

array([True, True, True, ..., False, False, False], dtype=object)

In [63]:
y = [int(tag is True) for tag in y]

In [64]:
y = np.array(y)

In [65]:
y = 2*y-1

In [66]:
C = 1000
m, n = X.shape

y = y.reshape(-1,1) * 1.
X_dash = y * X
H = np.dot(X_dash , X_dash.T) * 1.

In [67]:
#Converting into cvxopt format - as previously
P = cvxopt_matrix(H)
q = cvxopt_matrix(-np.ones((m, 1)))
G = cvxopt_matrix(np.vstack((np.eye(m)*-1,np.eye(m))))
h = cvxopt_matrix(np.hstack((np.zeros(m), np.ones(m) * C)))
A = cvxopt_matrix(y.reshape(1, -1))
b = cvxopt_matrix(np.zeros(1))

#Run solver
sol = cvxopt_solvers.qp(P, q, G, h, A, b)
alphas = np.array(sol['x'])

     pcost       dcost       gap    pres   dres
 0: -1.2799e+06 -5.6973e+08  6e+08  2e-11  3e-09
 1: -1.5992e+06 -9.4829e+06  8e+06  6e-10  4e-09
 2: -2.1150e+06 -3.2279e+06  1e+06  4e-10  4e-09
 3: -2.6129e+06 -3.2025e+06  6e+05  8e-10  4e-09
 4: -2.8551e+06 -3.1556e+06  3e+05  1e-10  5e-09
 5: -2.9569e+06 -3.1372e+06  2e+05  4e-10  5e-09
 6: -3.0173e+06 -3.1196e+06  1e+05  2e-10  5e-09
 7: -3.0566e+06 -3.1059e+06  5e+04  9e-10  6e-09
 8: -3.0739e+06 -3.0996e+06  3e+04  2e-10  6e-09
 9: -3.0829e+06 -3.0957e+06  1e+04  3e-10  6e-09
10: -3.0895e+06 -3.0915e+06  2e+03  1e-10  7e-09
11: -3.0902e+06 -3.0908e+06  5e+02  1e-09  6e-09
12: -3.0905e+06 -3.0905e+06  7e+01  1e-10  6e-09
13: -3.0905e+06 -3.0905e+06  6e+00  6e-10  7e-09
14: -3.0905e+06 -3.0905e+06  7e-02  2e-11  7e-09
Optimal solution found.


In [68]:
w = ((y * alphas).T @ X).reshape(-1,1)
S = (alphas > 1e-4).flatten()
b = y[S] - np.dot(X[S], w)

#Display results
print('Alphas = ',alphas[alphas > 1e-4])
print('w = ', w.flatten())
print('b = ', b[0])

Alphas =  [999.99999399 999.99999183 999.99999199 ... 999.99998349 999.99998898
 999.99997967]
w =  [ 0.67294253  0.          0.          0.         -0.44144201  0.
  0.          0.         -1.90071626  0.          0.          0.
 -0.04283892  0.          0.          0.          0.          0.29117317
  0.          0.          0.         -1.4200326   0.          0.
  0.          1.83463819  0.          0.          0.          1.00627589
  0.66901781  0.          0.          0.         -0.96820294  0.
  0.          0.         -2.18774571  0.          0.          0.
  2.20045184  0.          0.          0.          0.         -2.34467648
  0.          0.          0.          0.67685784  0.          0.
  0.          2.91241287  0.          0.          0.         -0.95811524]
b =  [1.3028943]


In [69]:
w.shape

(60, 1)

In [70]:
X.shape

(3197, 60)

In [71]:
svm = np.matmul(X, w) + b[0]

In [72]:
y_predic = np.where(svm > 0, 1, -1).flatten()

In [73]:
y_predic

array([1, 1, 1, ..., 1, 1, 1])

In [74]:
print(classification_report(y_pred=y_predic, y_true=y, labels=[-1,1]))

              precision    recall  f1-score   support

          -1       0.75      0.00      0.00      1598
           1       0.50      1.00      0.67      1599

    accuracy                           0.50      3197
   macro avg       0.63      0.50      0.34      3197
weighted avg       0.63      0.50      0.34      3197



In [75]:
np.unique(y_predic)

array([-1,  1])